In [1]:
import os
import re
import time
import pickle
from typing import Tuple, List

from smart_open import smart_open
from gensim.utils import simple_preprocess
import spacy
import multiprocessing
#from gensim import 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy import sparse
import gensim
from gensim.corpora import Dictionary, MmCorpus
from collections import deque
from collections import namedtuple
import random
from gensim.models.coherencemodel import CoherenceModel
# explore the topics discussed in the reviews and compare how they have evolved over time
# We will define a series of functions to achieve that objective
from gensim.models import phrases
from smart_open import smart_open
from gensim.utils import simple_preprocess
import os

In [ ]:
# read the reviews
pitchfork = pd.read_csv('../data/pitchfork/pitchfork.csv', low_memory=False, 
                       encoding = 'utf-8')
pitchfork.head()

In [ ]:
# read the stopwords
with open('../data/pitchfork/stop.txt', 'r') as f:
    stop_words = f.read().split('\n')
stop_words = stop_words+['songs', 'albums','record','records', 'album', 'sound', 'music',
                        'band', 'song', 'time', 'years','fuck', '\\xa0', 'kind','single','shit',
                         'good', '\n', 'great','\xa0', 'sing','rock','bad','guy','lyric',
                        'lot', 'sing','band','rock','man','girl','listen','day','bands','record',
                         'records', 'guitars','thing','pretty','artist','things',
                         'people','stuff']
#stop_words = set(stop_words)

In [ ]:
#pitchfork['review'] = pitchfork['review'].values.astype('str')
documents = pitchfork['review'].tolist()
print(len(documents))

In [ ]:
documents = [str(doc) for doc in documents if doc]

In [ ]:
print(len(documents))
documents = [doc for doc in documents if len(doc)>200]
print(len(documents))

In [2]:
# load dictionary
dictionary = Dictionary.load('../data/pitchfork/dict.pkl')
bows = gensim.corpora.MmCorpus('../data/pitchfork/corpus.mm')
docs = pickle.load(open('../data/pitchfork/docs.pkl','rb'))
print('len of vocabulary is ',len(dictionary))

len of vocabulary is  7291


In [3]:
# split into training and test
from sklearn.model_selection import train_test_split
x_bows_train, x_bows_test = train_test_split(bows, test_size=0.15, random_state=192)
x_tokens_train, x_tokens_test = train_test_split(docs, test_size=0.15, random_state=192)

In [4]:
#item = list(zip(*bows[0]))
#torch.tensor(list(item[1])).float()
print(f'len of training size is {len(x_bows_train)} and tokes are {len(x_tokens_train)} '
      f'len of test size is {len(x_bows_test)} and tokens are {len(x_tokens_test)})')

len of training size is 5757 and tokes are 5757 len of test size is 1017 and tokens are 1017)


In [5]:
from torch.utils.data import Dataset,DataLoader
import torch

class Data_Processing(object):
    def __init__(self, docs, bows, vocab):
        self.docs = docs
        self.bows = bows
        self.vocab = vocab
        
    def __getitem__(self,idx):
        bow = torch.zeros(len(self.vocab))
        # create token and frequency
        item = list(zip(*self.bows[idx])) # bow = [[token_id1,token_id2,...],[freq1,freq2,...]]
        # create
        bow[list(item[0])] = torch.tensor(list(item[1])).float()
        txt = self.docs[idx]
        #print(f'shape of bow before being put together in data loader {bow.shape} {type(bow)}')
        return txt, bow
    
    def __len__(self):
        return len(self.docs)
    
    def collate_fn1(self,batch_data):
        texts,bows = list(zip(*batch_data))
        #print(f'what happens with collate function {torch.stack(bows,dim=0)}, {torch.stack(bows,dim=0).shape}')
        return texts,torch.stack(bows,dim=0)

    def __iter__(self):
        for doc in self.docs:
            yield doc

batch_size = 512

# create a class to process the traininga and test data
training_data = Data_Processing(x_tokens_train, x_bows_train, dictionary)
test_data = Data_Processing(x_tokens_test, x_bows_test, dictionary)

# use the dataloaders class to load the data
dataloaders_dict = {'train': DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=4,
                                        collate_fn=training_data.collate_fn1),
                    'test': DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4,
                                        collate_fn=test_data.collate_fn1)}
dataset_sizes = {'train':len(training_data)}
example = next(iter(dataloaders_dict.get('train')))

In [6]:
example[1][0].shape

torch.Size([7291])

In [7]:
# define the model
import torch
import torch.nn.functional as F
from torch import nn


class ETM(nn.Module):
    # instantiate the model
    def __init__(self, device = None, num_topics = 20, vocab_size = None, t_hidden_size = 1024, 
                 rho_size = 300, emb_size=300, theta_act='relu',
                 embeddings=None, train_embeddings=True, enc_drop=0.5, debug_mode=False):
        '''
        Creates an embedded topic model instance. The model hyperparameters are:
        device: if using cuda or cpu
        num_topics: number of topics to be used
        vocab_size: size of the vocabulary
        t_hidden_size:  
        embeddings (str or KeyedVectors): KeyedVectors instance containing word-vector mapping for embeddings, 
                                          or its path
        use_c_format_w2vec (bool): wheter input embeddings use word2vec C format. Both BIN and TXT formats are 
                                   supported
        model_path (str): path to save trained model. If None, the model won't be automatically saved
        batch_size (int): input batch size for training
        num_topics (int): number of topics
        rho_size (int): dimension of rho
        emb_size (int): dimens-ion of embeddingsfno
        t_hidden_size (int): dimension of hidden space of q(theta)
        theta_act (str): activation function can be tanh, softplus, relu, rrelu, leakyrelu, elu, selu, glu, etc.
        train_embeddings (bool): whether to fix rho or train it, defaults to True
        
        # separate this part
        lr (float): learning rate
        lr_factor (float): divide learning rate by this...
        epochs (int): number of epochs to train. 150 for 20ng 100 for others
        optimizer_type (str): choice of optimizer
        seed (int): random seed (default: 1)
        enc_drop (float): dropout rate on encoder
        clip (float): gradient clipping
        nonmono (int): number of bad hits allowed
        wdecay (float): some l2 regularization
        anneal_lr (bool): whether to anneal the learning rate or not
        bow_norm (bool): normalize the bows or not
        num_words (int): number of words for topic viz
        log_interval (int): when to log training
        visualize_every (int): when to visualize results
        eval_batch_size (int): input batch size for evaluation
        eval_perplexity (bool): whether to compute perplexity on document completion task
        debug_mode (bool): wheter or not should log model operations
        '''
        super(ETM, self).__init__()
        # define hyperparameters
        self.num_topics = num_topics
        self.vocab_size = vocab_size
        # in case we want to train the embeddings this parameter controls the hidden size of the embeddings
        self.rho_size = rho_size
        # dimension of the matrix for the topic mixture (topics hidden size)
        self.t_hidden_size = t_hidden_size
        # dropout to be applied to the encoder
        self.enc_drop = enc_drop
        # size of the embeddings in case a pretrained embedding is provided
        self.emb_size = emb_size
        # dropout rate
        self.t_drop = nn.Dropout(enc_drop)
        self.debug_mode = debug_mode
        # activation for theta
        self.theta_act = self.get_activation(theta_act)
        # device to use for training
        self.device = device

        # define the word embedding matrix \rho
        if train_embeddings:
            self.rho = nn.Linear(rho_size, vocab_size, bias=True)
        else:
            # get the dimension of the embeddings
            print(embeddings.size())
            num_embeddings, emb_size = embeddings.size()
            # instantiate rho embeddings
            self.rho = embeddings.clone().float().to(self.device)

        # define the matrix containing the topic embeddings
        self.alphas = nn.Linear(rho_size, num_topics, bias=False)

        # define variational distribution for \theta_{1:D} via amortization
        # this takes the vocabulary size, creates the hidden size for the
        # topics matrix, applies activation function, applies another linear
        # layer followed by additional activation function
        self.q_theta = nn.Sequential(nn.Linear(vocab_size, t_hidden_size),
            self.theta_act,
            nn.Linear(t_hidden_size, t_hidden_size),
            self.theta_act,
        )
        
        # linear transformation of the topics hidden size to the number of topics
        self.mu_q_theta = nn.Linear(t_hidden_size, num_topics, bias=True)
        
        # another linear transformation from hidden size to number of topics
        self.logsigma_q_theta = nn.Linear(t_hidden_size, num_topics, bias=True)

    def get_activation(self, act = 'relu'):
        '''
        Define the activation function options available are tahn, relu, softplus, rrelu, 
        leakyrelu, elu, selu and glu.
        '''
        if act == 'tanh':
            act = nn.Tanh()
        elif act == 'relu':
            act = nn.ReLU()
        elif act == 'softplus':
            act = nn.Softplus()
        elif act == 'rrelu':
            act = nn.RReLU()
        elif act == 'leakyrelu':
            act = nn.LeakyReLU()
        elif act == 'elu':
            act = nn.ELU()
        elif act == 'selu':
            act = nn.SELU()
        elif act == 'glu':
            act = nn.GLU()
        else:
            act = nn.Tanh()
            if self.debug_mode:
                print('Defaulting to tanh activation')
        return act

    def reparameterize(self, mu, logvar):
        """
        Returns a sample from a Gaussian distribution via reparameterization.
        mu stands for
        logvar stands for 
        reference for reparametrizetion trick. 
        https://stats.stackexchange.com/questions/199605/how-does-the-reparameterization-trick-for-vaes-work-and-why-is-it-important
        """
        if self.training:
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            return eps.mul_(std).add_(mu)
        else:
            return mu

    def encode(self, bows):
        """Returns paramters of the variational distribution for \theta.
        input: bows batch of bag-of-words...tensor of shape bsz x V (batch size* vocabulary)
        output: mu_theta, log_sigma_theta
        """
        
        q_theta = self.q_theta(bows)
        if self.enc_drop > 0:
            q_theta = self.t_drop(q_theta)
        # mu_q_theta takes the topic hidden size and outputs a vector with the dimension of the topics
        mu_theta = self.mu_q_theta(q_theta)
        # takes the topic hidden size and outpus another vector equal to the size of the vectors
        logsigma_theta = self.logsigma_q_theta(q_theta)
        # returns kl theta which is a normalized number
        kl_theta = -0.5 * torch.sum(1 + logsigma_theta - mu_theta.pow(2) - logsigma_theta.exp(), dim=-1).mean()
        #print(f'this is kl_theta {kl_theta}')
        return mu_theta, logsigma_theta, kl_theta
       
    def get_beta(self):
        """
        This generate the description as a defintion over words
        Returns:
            [type]: [description]
        """
        try:
            logit = self.alphas(self.rho.weight) # torch.mm(self.rho, self.alphas)
        except:
            logit = self.alphas(self.rho)
        # gets logits over vocabulary dimension when fitting to the topic
        beta = F.softmax(logit, dim=0).transpose(1, 0) ## softmax over vocab dimension
        #print(f'this is beta {beta}')
        return beta

    def get_theta(self, normalized_bows):
        """
        getting the topic poportion for the document passed in the normalixe bow or tf-idf
        """
        # gets mu_theta (linear layer for topic dimension), logsima_theta another liner layer output
        # and the kld_tjet
        mu_theta, logsigma_theta, kld_theta = self.encode(normalized_bows)
        # gets variable to backpropagate, returns  reparametized mu theta
        z = self.reparameterize(mu_theta, logsigma_theta)
        # applied softmax and returns squeezed logits
        theta = F.softmax(z, dim=-1) 
        #print(f'this is theta {theta}')
        return theta, kld_theta

    def decode(self, theta, beta):
        """compute the probability of topic given the document which is equal to theta^T ** B
        Args:
            theta ([type]): vector of proabilities of belonging to a given topic
            beta ([type]): probability of belonging to a given topic
        Returns:
            [type]: [description]
        """
        # multiplication of softmax with beta (likelihood of belonging to a topic given likelihood a word
        # belongs to another topic)
        res = torch.mm(theta, beta)
        # normalizes predictions
        almost_zeros = torch.full_like(res, 1e-6)
        results_without_zeros = res.add(almost_zeros)
        predictions = torch.log(results_without_zeros)
        return predictions

    def forward(self, bows, normalized_bows, theta=None, aggregate=True):
        # get \theta
        if theta is None:
            theta, kld_theta = self.get_theta(normalized_bows)
        else:
            kld_theta = None

        # get \beta
        beta = self.get_beta()

        # get prediction loss
        preds = self.decode(theta, beta)
        recon_loss = -(preds * bows).sum(1)
        if aggregate:
            recon_loss = recon_loss.mean()
        return recon_loss, kld_theta

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

In [9]:
model = ETM(device = device, num_topics = 20, vocab_size =len(dictionary), 
            t_hidden_size = 1024, rho_size = 300, 
            emb_size=300, theta_act='tahn',
            embeddings=None, train_embeddings=True, enc_drop=0.4, debug_mode=False)

In [10]:
model

ETM(
  (t_drop): Dropout(p=0.4, inplace=False)
  (theta_act): Tanh()
  (rho): Linear(in_features=300, out_features=7291, bias=True)
  (alphas): Linear(in_features=300, out_features=20, bias=False)
  (q_theta): Sequential(
    (0): Linear(in_features=7291, out_features=1024, bias=True)
    (1): Tanh()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): Tanh()
  )
  (mu_q_theta): Linear(in_features=1024, out_features=20, bias=True)
  (logsigma_q_theta): Linear(in_features=1024, out_features=20, bias=True)
)

In [11]:
#def expand_array(vocab,index_array,values_array):
#    a = np.zeros(len(vocab))
#    a[train_dataset['tokens'][0]] = [x for x in train_dataset['counts'][0].tolist()]
#    return a


In [12]:
from torch.optim import lr_scheduler, AdamW, SGD,Adam
lr = 0.003
optimizer = Adam(model.parameters(), lr=lr, weight_decay = 0.001)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.003
    weight_decay: 0.001
)

In [13]:
#model.forward()
def get_topics(model = None, num_topics = 25, top_n_words=10, vocabulary = None) -> List[str]:
        """
        Gets topics. By default, returns the 10 most relevant terms for each topic.
        Parameters:
        ===
            top_n_words (int): number of top words per topic to return
        Returns:
        ===
            list of str: topic list
        """

        with torch.no_grad():
            topics = []
            gammas = model.get_beta()

            for k in range(model.num_topics):
                gamma = gammas[k]
                top_words = list(gamma.cpu().numpy().argsort()
                                 [-top_n_words:][::-1])
                topic_words = [vocabulary[a] for a in top_words]
                topics.append(topic_words)

            return topics

def get_topic_words(model,dictionary):
    with torch.no_grad():
        topic_words = []
        gammas = model.get_beta()
        
        for k in range(model.num_topics):
            gamma = gammas[k]
            top_words = list(gamma.cpu().numpy().argsort()[::-1])
            tp = [dictionary[a] for a in top_words]
            topic_words.append(tp)
    
    return topic_words
        
def calc_topic_diversity(topic_words):
    '''topic_words is in the form of [[w11,w12,...],[w21,w22,...]]'''
    vocab = set(sum(topic_words,[]))
    n_total = len(topic_words) * len(topic_words[0])
    print(len(topic_words))
    topic_div = len(vocab) / n_total
    return topic_div


def get_topic_diversity(model, topk=50):
    with torch.no_grad():
        beta = model.get_beta()
        num_topics = model.num_topics
        print(beta.shape, num_topics)
        list_w = np.zeros((num_topics, topk))
        print(list_w.shape)
        for k in range(num_topics):
            idx = beta[k,:].cpu().numpy().argsort()[-topk:][::-1]
            list_w[k,:] = idx
        n_unique = len(np.unique(list_w))
        TD = n_unique / (topk * num_topics)
        #print('Topic diveristy is: {}'.format(TD))
        return TD

In [14]:
from pprint import pprint
def nearest_neighbors(word, embeddings, vocab, n_most_similar=5):
    vectors = embeddings.data.cpu().numpy()
    index = dictionary.token2id[word]
    query = vectors[index]
    ranks = vectors.dot(query).squeeze()
    denom = query.T.dot(query).squeeze()
    denom = denom * np.sum(vectors**2, 1)
    denom = np.sqrt(denom)
    ranks = ranks / denom
    mostSimilar = []
    [mostSimilar.append(idx) for idx in ranks.argsort()[::-1]]
    nearest_neighbors = mostSimilar[0:n_most_similar]
    nearest_neighbors = [vocab[comp] for comp in nearest_neighbors]
    return nearest_neighbors

def get_most_similar_words(model = None, queries=[], vocabulary = None, n_most_similar=5) -> dict:
        """
        Gets the nearest neighborhoring words for a list of tokens. By default, returns the 20 most similar words for each token in 'queries' array.
        Parameters:
        ===
            queries (list of str): words to find similar ones
            n_most_similar (int): number of most similar words to get for each word given in the input. By default is 20
        Returns:
        ===
            dict of (str, list of str): dictionary containing the mapping between query words given and their respective similar words
        """

        model.eval()

        # visualize word embeddings by using V to get nearest neighbors
        with torch.no_grad():
            embeddings = model.rho.weight  # Vocab_size x E
            

            neighbors = {}
            for word in queries:
                neighbors[word] = nearest_neighbors(
                    word,embeddings, vocabulary, n_most_similar)

            return neighbors

In [15]:
#import wandb
#wandb.init(entity="jlealtru", project="ETM_runs_p")
#config = wandb.config          # Initialize config

def train(model, batch_size=256,dictionary = None,
          learning_rate=2e-3,test_data=None,
          num_epochs=600,is_evaluate=False,log_every=40,ckpt=None):
    model.to(device)
    model.train()
    
    data_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=4,
                                        collate_fn=training_data.collate_fn1)
    
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
    
    
    if ckpt:
        self.load_model(ckpt["net"])
        optimizer.load_state_dict(ckpt["optimizer"])
        start_epoch = ckpt["epoch"] + 1
    else:
        start_epoch = 0
    
    acc_loss = 0
    acc_kl_theta_loss = 0
    cnt = 0
    
    trainloss_lst, valloss_lst = [], []
    recloss_lst, klloss_lst = [],[]
    c_v_lst, c_w2v_lst, c_uci_lst, c_npmi_lst, mimno_tc_lst, td_lst = [], [], [], [], [], []
    for epoch in range(start_epoch, num_epochs):
        epochloss_lst = []
        model.train()
        for iter_,data in enumerate(data_loader):
            #optimizer.zero_grad()
            model.zero_grad(set_to_none=True)
            
            txts,bows = data
            bows = bows.to(device)
            normalized_bows = bows
            normalized_bows.to(device)
            
            recon_loss, kld_theta = model.forward(bows, normalized_bows)
            total_loss = recon_loss + kld_theta
            total_loss.backward()
            optimizer.step()

            acc_loss += torch.sum(recon_loss).item()
            acc_kl_theta_loss += torch.sum(kld_theta).item()
            cnt += 1
            if iter_ % 4 == 0:
                cur_loss = round(acc_loss / cnt, 2) 
                cur_kl_theta = round(acc_kl_theta_loss / cnt, 2) 
                cur_real_loss = round(cur_loss + cur_kl_theta, 2)

                print('Epoch: {} KL_theta: is {} .. Rec_loss: {} .. NELBO: {}'.format(
                    epoch, cur_kl_theta, cur_loss, cur_real_loss))

        cur_loss = round(acc_loss / cnt, 2) 
        cur_kl_theta = round(acc_kl_theta_loss / cnt, 2) 
        cur_real_loss = round(cur_loss + cur_kl_theta, 2)
        print('*'*100)
        print('Epoch: {} KL_theta: is {} .. Rec_loss: {} .. NELBO: {}'.format(
                    epoch, cur_kl_theta, cur_loss, cur_real_loss))
        if (epoch+1)%40==0:
            topic_words = get_topic_words(model, dictionary)
            topic_diversity = get_topic_diversity(model,topk=100)
            #calc_topic_diversity(topic_words)
            print(f'topic diversity is {topic_diversity}')
            pprint(get_topics(model = model, num_topics = 25, top_n_words= 10, vocabulary = dictionary))

In [16]:
#model.to(device)
#wandb.watch(model, log="all")
train(model,batch_size=1024, learning_rate=2e-3,test_data=None,dictionary=dictionary,
      num_epochs=200,is_evaluate=False,log_every=40,
      ckpt=None)

Epoch: 0 KL_theta: is 0.04 .. Rec_loss: 1501.15 .. NELBO: 1501.19
Epoch: 0 KL_theta: is 6.03 .. Rec_loss: 1516.79 .. NELBO: 1522.82
****************************************************************************************************
Epoch: 0 KL_theta: is 5.2 .. Rec_loss: 1514.48 .. NELBO: 1519.68
Epoch: 1 KL_theta: is 4.79 .. Rec_loss: 1514.88 .. NELBO: 1519.67
Epoch: 1 KL_theta: is 3.74 .. Rec_loss: 1511.65 .. NELBO: 1515.39
****************************************************************************************************
Epoch: 1 KL_theta: is 3.58 .. Rec_loss: 1510.93 .. NELBO: 1514.51
Epoch: 2 KL_theta: is 3.36 .. Rec_loss: 1510.68 .. NELBO: 1514.04
Epoch: 2 KL_theta: is 2.79 .. Rec_loss: 1505.0 .. NELBO: 1507.79
****************************************************************************************************
Epoch: 2 KL_theta: is 2.67 .. Rec_loss: 1504.8 .. NELBO: 1507.47
Epoch: 3 KL_theta: is 2.54 .. Rec_loss: 1504.37 .. NELBO: 1506.91
Epoch: 3 KL_theta: is 2.17 .. Rec_loss: 

****************************************************************************************************
Epoch: 27 KL_theta: is 0.39 .. Rec_loss: 1445.75 .. NELBO: 1446.14
Epoch: 28 KL_theta: is 0.39 .. Rec_loss: 1445.6 .. NELBO: 1445.99
Epoch: 28 KL_theta: is 0.38 .. Rec_loss: 1445.39 .. NELBO: 1445.77
****************************************************************************************************
Epoch: 28 KL_theta: is 0.38 .. Rec_loss: 1445.45 .. NELBO: 1445.83
Epoch: 29 KL_theta: is 0.38 .. Rec_loss: 1445.32 .. NELBO: 1445.7
Epoch: 29 KL_theta: is 0.38 .. Rec_loss: 1445.21 .. NELBO: 1445.59
****************************************************************************************************
Epoch: 29 KL_theta: is 0.37 .. Rec_loss: 1445.11 .. NELBO: 1445.48
Epoch: 30 KL_theta: is 0.37 .. Rec_loss: 1445.09 .. NELBO: 1445.46
Epoch: 30 KL_theta: is 0.37 .. Rec_loss: 1444.87 .. NELBO: 1445.24
************************************************************************************************

Epoch: 47 KL_theta: is 0.34 .. Rec_loss: 1441.65 .. NELBO: 1441.99
Epoch: 47 KL_theta: is 0.34 .. Rec_loss: 1441.58 .. NELBO: 1441.92
****************************************************************************************************
Epoch: 47 KL_theta: is 0.34 .. Rec_loss: 1441.52 .. NELBO: 1441.86
Epoch: 48 KL_theta: is 0.34 .. Rec_loss: 1441.5 .. NELBO: 1441.84
Epoch: 48 KL_theta: is 0.34 .. Rec_loss: 1441.39 .. NELBO: 1441.73
****************************************************************************************************
Epoch: 48 KL_theta: is 0.34 .. Rec_loss: 1441.4 .. NELBO: 1441.74
Epoch: 49 KL_theta: is 0.34 .. Rec_loss: 1441.33 .. NELBO: 1441.67
Epoch: 49 KL_theta: is 0.35 .. Rec_loss: 1441.26 .. NELBO: 1441.61
****************************************************************************************************
Epoch: 49 KL_theta: is 0.35 .. Rec_loss: 1441.29 .. NELBO: 1441.64
Epoch: 50 KL_theta: is 0.35 .. Rec_loss: 1441.2 .. NELBO: 1441.55
Epoch: 50 KL_theta: is 0.35 ..

Epoch: 74 KL_theta: is 0.86 .. Rec_loss: 1437.97 .. NELBO: 1438.83
****************************************************************************************************
Epoch: 74 KL_theta: is 0.86 .. Rec_loss: 1437.93 .. NELBO: 1438.79
Epoch: 75 KL_theta: is 0.87 .. Rec_loss: 1437.87 .. NELBO: 1438.74
Epoch: 75 KL_theta: is 0.88 .. Rec_loss: 1437.79 .. NELBO: 1438.67
****************************************************************************************************
Epoch: 75 KL_theta: is 0.89 .. Rec_loss: 1437.81 .. NELBO: 1438.7
Epoch: 76 KL_theta: is 0.89 .. Rec_loss: 1437.79 .. NELBO: 1438.68
Epoch: 76 KL_theta: is 0.91 .. Rec_loss: 1437.7 .. NELBO: 1438.61
****************************************************************************************************
Epoch: 76 KL_theta: is 0.91 .. Rec_loss: 1437.68 .. NELBO: 1438.59
Epoch: 77 KL_theta: is 0.91 .. Rec_loss: 1437.65 .. NELBO: 1438.56
Epoch: 77 KL_theta: is 0.93 .. Rec_loss: 1437.61 .. NELBO: 1438.54
*****************************

Epoch: 94 KL_theta: is 1.29 .. Rec_loss: 1435.53 .. NELBO: 1436.82
Epoch: 94 KL_theta: is 1.31 .. Rec_loss: 1435.52 .. NELBO: 1436.83
****************************************************************************************************
Epoch: 94 KL_theta: is 1.31 .. Rec_loss: 1435.46 .. NELBO: 1436.77
Epoch: 95 KL_theta: is 1.31 .. Rec_loss: 1435.43 .. NELBO: 1436.74
Epoch: 95 KL_theta: is 1.33 .. Rec_loss: 1435.4 .. NELBO: 1436.73
****************************************************************************************************
Epoch: 95 KL_theta: is 1.33 .. Rec_loss: 1435.32 .. NELBO: 1436.65
Epoch: 96 KL_theta: is 1.34 .. Rec_loss: 1435.3 .. NELBO: 1436.64
Epoch: 96 KL_theta: is 1.35 .. Rec_loss: 1435.22 .. NELBO: 1436.57
****************************************************************************************************
Epoch: 96 KL_theta: is 1.35 .. Rec_loss: 1435.21 .. NELBO: 1436.56
Epoch: 97 KL_theta: is 1.36 .. Rec_loss: 1435.17 .. NELBO: 1436.53
Epoch: 97 KL_theta: is 1.37 .

Epoch: 120 KL_theta: is 1.89 .. Rec_loss: 1432.78 .. NELBO: 1434.67
Epoch: 120 KL_theta: is 1.9 .. Rec_loss: 1432.73 .. NELBO: 1434.63
****************************************************************************************************
Epoch: 120 KL_theta: is 1.91 .. Rec_loss: 1432.68 .. NELBO: 1434.59
Epoch: 121 KL_theta: is 1.91 .. Rec_loss: 1432.65 .. NELBO: 1434.56
Epoch: 121 KL_theta: is 1.93 .. Rec_loss: 1432.58 .. NELBO: 1434.51
****************************************************************************************************
Epoch: 121 KL_theta: is 1.93 .. Rec_loss: 1432.58 .. NELBO: 1434.51
Epoch: 122 KL_theta: is 1.93 .. Rec_loss: 1432.54 .. NELBO: 1434.47
Epoch: 122 KL_theta: is 1.95 .. Rec_loss: 1432.48 .. NELBO: 1434.43
****************************************************************************************************
Epoch: 122 KL_theta: is 1.96 .. Rec_loss: 1432.47 .. NELBO: 1434.43
Epoch: 123 KL_theta: is 1.96 .. Rec_loss: 1432.43 .. NELBO: 1434.39
Epoch: 123 KL_thet

Epoch: 147 KL_theta: is 2.54 .. Rec_loss: 1429.98 .. NELBO: 1432.52
Epoch: 147 KL_theta: is 2.56 .. Rec_loss: 1429.91 .. NELBO: 1432.47
****************************************************************************************************
Epoch: 147 KL_theta: is 2.56 .. Rec_loss: 1429.89 .. NELBO: 1432.45
Epoch: 148 KL_theta: is 2.57 .. Rec_loss: 1429.87 .. NELBO: 1432.44
Epoch: 148 KL_theta: is 2.58 .. Rec_loss: 1429.81 .. NELBO: 1432.39
****************************************************************************************************
Epoch: 148 KL_theta: is 2.58 .. Rec_loss: 1429.79 .. NELBO: 1432.37
Epoch: 149 KL_theta: is 2.59 .. Rec_loss: 1429.78 .. NELBO: 1432.37
Epoch: 149 KL_theta: is 2.6 .. Rec_loss: 1429.71 .. NELBO: 1432.31
****************************************************************************************************
Epoch: 149 KL_theta: is 2.61 .. Rec_loss: 1429.69 .. NELBO: 1432.3
Epoch: 150 KL_theta: is 2.61 .. Rec_loss: 1429.68 .. NELBO: 1432.29
Epoch: 150 KL_theta

Epoch: 166 KL_theta: is 2.97 .. Rec_loss: 1428.19 .. NELBO: 1431.16
Epoch: 166 KL_theta: is 2.99 .. Rec_loss: 1428.1 .. NELBO: 1431.09
****************************************************************************************************
Epoch: 166 KL_theta: is 2.99 .. Rec_loss: 1428.11 .. NELBO: 1431.1
Epoch: 167 KL_theta: is 3.0 .. Rec_loss: 1428.07 .. NELBO: 1431.07
Epoch: 167 KL_theta: is 3.01 .. Rec_loss: 1428.03 .. NELBO: 1431.04
****************************************************************************************************
Epoch: 167 KL_theta: is 3.01 .. Rec_loss: 1428.01 .. NELBO: 1431.02
Epoch: 168 KL_theta: is 3.02 .. Rec_loss: 1427.98 .. NELBO: 1431.0
Epoch: 168 KL_theta: is 3.03 .. Rec_loss: 1427.92 .. NELBO: 1430.95
****************************************************************************************************
Epoch: 168 KL_theta: is 3.04 .. Rec_loss: 1427.93 .. NELBO: 1430.97
Epoch: 169 KL_theta: is 3.04 .. Rec_loss: 1427.91 .. NELBO: 1430.95
Epoch: 169 KL_theta: 

Epoch: 193 KL_theta: is 3.57 .. Rec_loss: 1425.81 .. NELBO: 1429.38
Epoch: 193 KL_theta: is 3.59 .. Rec_loss: 1425.75 .. NELBO: 1429.34
****************************************************************************************************
Epoch: 193 KL_theta: is 3.59 .. Rec_loss: 1425.76 .. NELBO: 1429.35
Epoch: 194 KL_theta: is 3.59 .. Rec_loss: 1425.76 .. NELBO: 1429.35
Epoch: 194 KL_theta: is 3.61 .. Rec_loss: 1425.69 .. NELBO: 1429.3
****************************************************************************************************
Epoch: 194 KL_theta: is 3.61 .. Rec_loss: 1425.68 .. NELBO: 1429.29
Epoch: 195 KL_theta: is 3.62 .. Rec_loss: 1425.69 .. NELBO: 1429.31
Epoch: 195 KL_theta: is 3.63 .. Rec_loss: 1425.61 .. NELBO: 1429.24
****************************************************************************************************
Epoch: 195 KL_theta: is 3.63 .. Rec_loss: 1425.6 .. NELBO: 1429.23
Epoch: 196 KL_theta: is 3.64 .. Rec_loss: 1425.57 .. NELBO: 1429.21
Epoch: 196 KL_theta

In [17]:
# get topic proportion
#with model.eval():
#    model.get_theta(x_bows_train[0])
get_most_similar_words(model = model, queries=['interpol'], vocabulary = dictionary, n_most_similar=5)

{'interpol': ['interpol', 'spoon', 'nme', 'devo', 'ramones']}

In [44]:
# get topic words
get_topics(model = model, num_topics = 25, top_n_words= 15, vocabulary = dictionary)

[['rap',
  'hip_hop',
  'beats',
  'beat',
  'production',
  'rapper',
  'soul',
  'black',
  'style',
  'life',
  'flow',
  'producer',
  'rappers',
  'r&b',
  'raps'],
 ['group',
  'release',
  'ep',
  'interesting',
  'hear',
  'material',
  'fact',
  'point',
  'studio',
  'instrumental',
  'style',
  'minutes',
  'version',
  'musical',
  'play'],
 ['big',
  'days',
  'indie',
  'fact',
  'singer',
  'chorus',
  'cover',
  'past',
  'songwriting',
  'singing',
  'life',
  'real',
  'hear',
  'home',
  'friends'],
 ['punk',
  'smith',
  'girls',
  'garage',
  'kids',
  'guys',
  'indie',
  'fun',
  'boys',
  'wave',
  'fucking',
  'post_punk',
  'hardcore',
  'group',
  'riffs'],
 ['dance',
  'house',
  'techno',
  'label',
  'electronic',
  'disco',
  'beats',
  'mix',
  'remix',
  'dub',
  'beat',
  'bass',
  'synths',
  'club',
  'electro'],
 ['folk',
  'acoustic',
  'strings',
  'acoustic_guitar',
  'piano',
  'country',
  'gentle',
  'violin',
  'sings',
  'johnson',
  'percus

In [45]:
with torch.no_grad():
    bow = torch.zeros(len(dictionary))
    item = list(zip(*x_bows_train[1])) # bow = [[token_id1,token_id2,...],[freq1,freq2,...]]
    bow[list(item[0])] = torch.tensor(list(item[1])).float()
    topics = model.get_theta(torch.tensor(bow).float().to(device))
    print(f'topic is: {torch.argmax(topics[0])} and text is {x_tokens_train[1]}')

topic is: 19 and text is ['half', 'manner', 'david_bowie', 'inauspicious', 'kick', 'sentimental', 'start', 'life', 'mars', 'rest', 'performance', 'vh1', 'storytellers', 'series', 'poses', 'question', 'accurately', 'suggests', 'audience', 'privy', 'inside', 'information', 'answers', 'tepid', 'set', 'broadcast', 'august', 'quarter_century', 'hung', 'ziggy_stardust', 'kimonos', 'episode', 'teased', 'fans', 'rare', 'chance_hear', 'shape_shifting', 'discuss', 'deconstruct', 'mythology', 'bowie', 'running', 'fumes', 'disappointing', 'decade--', 'earthling', 'forthcoming', 'hours', 'iconic', 'catalog', 'colorful', 'personal', 'narrative', 'narratives', 'amazing', 'supporting_cast', 'draw', 'stories', 'bowie', 'sadly', 'flashes', 'wit', 'gregarious', 'storytelling', 'cd_dvd', 'release', 'television', 'performance', 'standing', 'center', 'community', 'theater', 'stage', 'set', 'entertaining', 'overly', 'polite', 'audience', 'donning', 'gray', 'hoodie', 'bowie', 'likable', 'cuff', 'introduction'

/home/jlealtru/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [37]:
#topics
npp

(tensor([0.0196, 0.0201, 0.0202, 0.0298, 0.5075, 0.0374, 0.0338, 0.0219, 0.0173,
         0.0222, 0.0285, 0.0242, 0.0195, 0.0256, 0.0253, 0.0391, 0.0240, 0.0252,
         0.0148, 0.0439], device='cuda:0'),
 tensor(7.2107, device='cuda:0'))

In [ ]:
get_most_similar_words(model = model, queries=['pop','cash'], vocabulary = vocab, n_most_similar=5)